In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 14799693895860110356, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17319388094061016771
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [10]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Xception_Dense201_descriptors"

In [11]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [12]:
# from keras.applications.vgg19 import VGG19
from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

In [13]:
input_tensor = Input(shape = input_shape)  

base_model1=Xception(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)
base_model2=DenseNet201(input_shape= input_shape,weights='imagenet', include_top=False, input_tensor=input_tensor)

x1 = base_model1.output
x1 = GlobalAveragePooling2D()(x1)

x2 = base_model2.output
x2 = GlobalAveragePooling2D()(x2)

merge = concatenate([x1, x2])
predictions = Dense(num_classes, activation='softmax')(merge)

model = Model(inputs=input_tensor,outputs=predictions)

In [14]:
for i, layer in enumerate(model.layers):
    print(i, layer.name)

0 input_1
1 zero_padding2d_1
2 conv1/conv
3 conv1/bn
4 conv1/relu
5 zero_padding2d_2
6 pool1
7 conv2_block1_0_bn
8 conv2_block1_0_relu
9 conv2_block1_1_conv
10 conv2_block1_1_bn
11 conv2_block1_1_relu
12 conv2_block1_2_conv
13 conv2_block1_concat
14 conv2_block2_0_bn
15 conv2_block2_0_relu
16 conv2_block2_1_conv
17 conv2_block2_1_bn
18 conv2_block2_1_relu
19 conv2_block2_2_conv
20 conv2_block2_concat
21 conv2_block3_0_bn
22 conv2_block3_0_relu
23 conv2_block3_1_conv
24 conv2_block3_1_bn
25 conv2_block3_1_relu
26 conv2_block3_2_conv
27 conv2_block3_concat
28 conv2_block4_0_bn
29 conv2_block4_0_relu
30 conv2_block4_1_conv
31 conv2_block4_1_bn
32 conv2_block4_1_relu
33 conv2_block4_2_conv
34 conv2_block4_concat
35 conv2_block5_0_bn
36 conv2_block5_0_relu
37 conv2_block5_1_conv
38 conv2_block5_1_bn
39 conv2_block5_1_relu
40 conv2_block5_2_conv
41 conv2_block5_concat
42 conv2_block6_0_bn
43 conv2_block6_0_relu
44 conv2_block6_1_conv
45 conv2_block6_1_bn
46 conv2_block6_1_relu
47 conv2_block

693 conv5_block23_0_bn
694 block7_sepconv2_act
695 conv5_block23_0_relu
696 block7_sepconv2
697 conv5_block23_1_conv
698 block7_sepconv2_bn
699 conv5_block23_1_bn
700 block7_sepconv3_act
701 conv5_block23_1_relu
702 block7_sepconv3
703 conv5_block23_2_conv
704 block7_sepconv3_bn
705 conv5_block23_concat
706 add_6
707 conv5_block24_0_bn
708 block8_sepconv1_act
709 conv5_block24_0_relu
710 block8_sepconv1
711 conv5_block24_1_conv
712 block8_sepconv1_bn
713 conv5_block24_1_bn
714 block8_sepconv2_act
715 conv5_block24_1_relu
716 block8_sepconv2
717 conv5_block24_2_conv
718 block8_sepconv2_bn
719 conv5_block24_concat
720 block8_sepconv3_act
721 conv5_block25_0_bn
722 block8_sepconv3
723 conv5_block25_0_relu
724 block8_sepconv3_bn
725 conv5_block25_1_conv
726 add_7
727 conv5_block25_1_bn
728 block9_sepconv1_act
729 conv5_block25_1_relu
730 block9_sepconv1
731 conv5_block25_2_conv
732 block9_sepconv1_bn
733 conv5_block25_concat
734 block9_sepconv2_act
735 conv5_block26_0_bn
736 block9_sepconv

In [15]:
c1 = model.layers[9].output 
c1 = GlobalAveragePooling2D()(c1)       

c2 = model.layers[55].output
c2 = GlobalAveragePooling2D()(c2)       

c3 = model.layers[143].output
c3 = GlobalAveragePooling2D()(c3)       

c4 = model.layers[483].output
c4 = GlobalAveragePooling2D()(c4) 

c5 = model.layers[609].output
c5 = GlobalAveragePooling2D()(c5) 

c6 = model.layers[631].output
c6 = GlobalAveragePooling2D()(c6) 

c7 = model.layers[647].output
c7 = GlobalAveragePooling2D()(c7) 

c8 = model.layers[825].output
c8 = GlobalAveragePooling2D()(c8) 

con = concatenate([c1, c2, c3, c4, c5, c6, c7, c8])

bottleneck_final_model = Model(inputs=model.input, outputs=con)

In [16]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [17]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [18]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [19]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [20]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [21]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [22]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [23]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [24]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_2': 0.99, 'lr': 0.01, 'beta_1': 0.7}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.1249 - acc: 0.5488 - val_loss: 1.0377 - val_acc: 0.5447
Epoch 2/15
 - 4s - loss: 0.9687 - acc: 0.6179 - val_loss: 0.9892 - val_acc: 0.5625
Epoch 3/15
 - 4s - loss: 0.9339 - acc: 0.6312 - val_loss: 0.9921 - val_acc: 0.5581
Epoch 4/15
 - 4s - loss: 0.9056 - acc: 0.6414 - val_loss: 0.9837 - val_acc: 0.5687
Epoch 5/15
 - 4s - loss: 0.8955 - acc: 0.6467 - val_loss: 0.9566 - val_acc: 0.5841
Epoch 6/15
 - 4s - loss: 0.8894 - acc: 0.6474 - val_loss: 0.9742 - val_acc: 0.5562
Epoch 7/15
 - 4s - loss: 0.8754 - acc: 0.6524 - val_loss: 0.9318 - val_acc: 0.5959
Epoch 8/15
 - 4s - loss: 0.8595 - acc: 0.6570 - val_loss: 0.9144 - val_acc: 0.6034
Epoch 9/15
 - 4s - loss: 0.8638 - acc: 0.6544 - val_loss: 0.9937 - val_acc: 0.5893
Epoch 10/15
 - 4s - loss: 0.8437 - acc: 0.6662 - val_loss: 0.9317 - val_acc: 0.6083
Epoch 11/15
 - 4s - loss: 0.8474 - acc: 0.6576 - val_l

Epoch 9/15
 - 4s - loss: 1.3996 - acc: 0.5297 - val_loss: 1.4904 - val_acc: 0.4091
Epoch 10/15
 - 4s - loss: 1.3694 - acc: 0.5422 - val_loss: 1.4711 - val_acc: 0.4145
Epoch 11/15
 - 4s - loss: 1.3401 - acc: 0.5528 - val_loss: 1.4513 - val_acc: 0.4207
Epoch 12/15
 - 4s - loss: 1.3105 - acc: 0.5672 - val_loss: 1.4306 - val_acc: 0.4248
Epoch 13/15
 - 4s - loss: 1.2840 - acc: 0.5776 - val_loss: 1.4125 - val_acc: 0.4295
Epoch 14/15
 - 4s - loss: 1.2576 - acc: 0.5871 - val_loss: 1.3944 - val_acc: 0.4322
Epoch 15/15
 - 4s - loss: 1.2355 - acc: 0.5940 - val_loss: 1.3756 - val_acc: 0.4354
14958/14958 [==============================] - 1s 34us/step
Validation Accuracy: 43.5352%
Validation Loss: 1.3756308529303345
Test Accuracy: 0.5676093680954485
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.4, 'lr': 1e-07, 'beta_1': 0.8}
Train on 34909 samples, validate on 1

Epoch 00014: early stopping
14958/14958 [==============================] - 1s 35us/step
Validation Accuracy: 68.5854%
Validation Loss: 0.8107499520058145
Test Accuracy: 0.8129326852261011
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.7, 'lr': 0.0001, 'beta_1': 0.6}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.5557 - acc: 0.3933 - val_loss: 1.4996 - val_acc: 0.4029
Epoch 2/15
 - 4s - loss: 1.3060 - acc: 0.5524 - val_loss: 1.3697 - val_acc: 0.4372
Epoch 3/15
 - 4s - loss: 1.1391 - acc: 0.6107 - val_loss: 1.2814 - val_acc: 0.4583
Epoch 4/15
 - 4s - loss: 1.0233 - acc: 0.6454 - val_loss: 1.1937 - val_acc: 0.4759
Epoch 5/15
 - 4s - loss: 0.9378 - acc: 0.6711 - val_loss: 1.1154 - val_acc: 0.5000
Epoch 6/15
 - 4s - loss: 0.8744 - acc: 0.6895 - val_loss: 1.0664 - val_acc: 0.5259
Epoch 7/15
 - 4s - loss: 0.8255 - acc: 0.7064 -

Parameters testing:  {'beta_2': 0.7, 'lr': 1e-05, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.7151 - acc: 0.2476 - val_loss: 1.6687 - val_acc: 0.2313
Epoch 2/15
 - 4s - loss: 1.6445 - acc: 0.2933 - val_loss: 1.6356 - val_acc: 0.2363
Epoch 3/15
 - 4s - loss: 1.6126 - acc: 0.3344 - val_loss: 1.6169 - val_acc: 0.2805
Epoch 4/15
 - 4s - loss: 1.5837 - acc: 0.3783 - val_loss: 1.6005 - val_acc: 0.3010
Epoch 5/15
 - 4s - loss: 1.5560 - acc: 0.4148 - val_loss: 1.5844 - val_acc: 0.3351
Epoch 6/15
 - 4s - loss: 1.5266 - acc: 0.4368 - val_loss: 1.5680 - val_acc: 0.3581
Epoch 7/15
 - 4s - loss: 1.5019 - acc: 0.4641 - val_loss: 1.5518 - val_acc: 0.3682
Epoch 8/15
 - 4s - loss: 1.4726 - acc: 0.4863 - val_loss: 1.5342 - val_acc: 0.3814
Epoch 9/15
 - 4s - loss: 1.4454 - acc: 0.5016 - val_loss: 1.5172 - val_acc: 0.3898
Epoch 10/15
 - 4s - loss: 1.4190 - acc: 0.5193 - val_loss: 1.5018 - val_acc: 0.3936
Epoch 11/15
 - 4s - loss: 1.3897 - acc: 0.5286 - val_l

Epoch 11/15
 - 4s - loss: 0.5184 - acc: 0.8137 - val_loss: 0.7957 - val_acc: 0.7020
Epoch 12/15
 - 4s - loss: 0.5067 - acc: 0.8177 - val_loss: 0.8025 - val_acc: 0.6964
Epoch 13/15
 - 4s - loss: 0.4962 - acc: 0.8216 - val_loss: 0.7659 - val_acc: 0.6893
Epoch 14/15
 - 4s - loss: 0.4940 - acc: 0.8252 - val_loss: 0.7947 - val_acc: 0.6754
Epoch 00014: early stopping
14958/14958 [==============================] - 1s 37us/step
Validation Accuracy: 67.5425%
Validation Loss: 0.7947420508908721
Test Accuracy: 0.7975401384592724
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.999, 'lr': 0.001, 'beta_1': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 6s - loss: 1.1168 - acc: 0.5884 - val_loss: 1.0399 - val_acc: 0.5522
Epoch 2/15
 - 4s - loss: 0.7711 - acc: 0.7198 - val_loss: 0.9639 - val_acc: 0.5873
Epoch 3/15
 - 4s - loss: 0.6789 - acc: 0.7

...

 - 13s - loss: 0.7683 - acc: 0.7189 - val_loss: 0.9277 - val_acc: 0.5989
Epoch 3/15
 - 14s - loss: 0.6826 - acc: 0.7578 - val_loss: 0.8980 - val_acc: 0.6322
Epoch 4/15
 - 13s - loss: 0.6315 - acc: 0.7742 - val_loss: 0.8654 - val_acc: 0.6488
Epoch 5/15
 - 11s - loss: 0.5986 - acc: 0.7868 - val_loss: 0.8498 - val_acc: 0.6750
Epoch 6/15
 - 14s - loss: 0.5769 - acc: 0.7927 - val_loss: 0.7915 - val_acc: 0.6959
Epoch 7/15
 - 14s - loss: 0.5583 - acc: 0.8017 - val_loss: 0.8357 - val_acc: 0.6501
Epoch 8/15
 - 14s - loss: 0.5411 - acc: 0.8090 - val_loss: 0.8181 - val_acc: 0.6818
Epoch 9/15
 - 14s - loss: 0.5261 - acc: 0.8123 - val_loss: 0.8436 - val_acc: 0.6530
Epoch 00009: early stopping
14958/14958 [==============================] - 2s 131us/step
Validation Accuracy: 65.3028%
Validation Loss: 0.8435739577952227
Test Accuracy: 0.7910590661363971
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*

Epoch 2/15
 - 14s - loss: 1.6447 - acc: 0.3010 - val_loss: 1.6371 - val_acc: 0.2562
Epoch 3/15
 - 14s - loss: 1.6092 - acc: 0.3433 - val_loss: 1.6161 - val_acc: 0.2855
Epoch 4/15
 - 13s - loss: 1.5769 - acc: 0.3919 - val_loss: 1.5973 - val_acc: 0.3095
Epoch 5/15
 - 13s - loss: 1.5467 - acc: 0.4222 - val_loss: 1.5789 - val_acc: 0.3329
Epoch 6/15
 - 15s - loss: 1.5168 - acc: 0.4569 - val_loss: 1.5590 - val_acc: 0.3742
Epoch 7/15
 - 14s - loss: 1.4833 - acc: 0.4793 - val_loss: 1.5405 - val_acc: 0.3816
Epoch 8/15
 - 12s - loss: 1.4537 - acc: 0.4980 - val_loss: 1.5207 - val_acc: 0.3958
Epoch 9/15
 - 14s - loss: 1.4227 - acc: 0.5145 - val_loss: 1.5008 - val_acc: 0.4059
Epoch 10/15
 - 14s - loss: 1.3912 - acc: 0.5257 - val_loss: 1.4826 - val_acc: 0.4110
Epoch 11/15
 - 15s - loss: 1.3637 - acc: 0.5385 - val_loss: 1.4637 - val_acc: 0.4176
Epoch 12/15
 - 12s - loss: 1.3380 - acc: 0.5520 - val_loss: 1.4439 - val_acc: 0.4241
Epoch 13/15
 - 14s - loss: 1.3125 - acc: 0.5588 - val_loss: 1.4266 - val_

Epoch 00004: early stopping
14958/14958 [==============================] - 3s 173us/step
Validation Accuracy: 20.7849%
Validation Loss: 1.7107391122796827
Test Accuracy: 0.27758138164678153
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_2': 0.99, 'lr': 0.001, 'beta_1': 0.9}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 25s - loss: 1.1809 - acc: 0.5615 - val_loss: 1.1204 - val_acc: 0.5055
Epoch 2/15
 - 15s - loss: 0.8251 - acc: 0.6974 - val_loss: 0.9503 - val_acc: 0.5892
Epoch 3/15
 - 10s - loss: 0.7316 - acc: 0.7334 - val_loss: 0.9246 - val_acc: 0.6395
Epoch 4/15
 - 13s - loss: 0.6854 - acc: 0.7562 - val_loss: 0.8580 - val_acc: 0.6635
Epoch 5/15
 - 13s - loss: 0.6466 - acc: 0.7696 - val_loss: 0.8759 - val_acc: 0.6351
Epoch 6/15
 - 14s - loss: 0.6239 - acc: 0.7802 - val_loss: 0.8106 - val_acc: 0.6708
Epoch 7/15
 - 15s - loss: 0.6020 - acc:

Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 24s - loss: 1.1259 - acc: 0.5857 - val_loss: 1.0843 - val_acc: 0.5380
Epoch 2/15
 - 13s - loss: 0.7713 - acc: 0.7189 - val_loss: 0.9227 - val_acc: 0.6013
Epoch 3/15
 - 13s - loss: 0.6805 - acc: 0.7519 - val_loss: 0.8905 - val_acc: 0.6301
Epoch 4/15
 - 13s - loss: 0.6325 - acc: 0.7715 - val_loss: 0.8289 - val_acc: 0.6796
Epoch 5/15
 - 12s - loss: 0.6013 - acc: 0.7856 - val_loss: 0.9049 - val_acc: 0.6372
Epoch 6/15
 - 15s - loss: 0.5753 - acc: 0.7941 - val_loss: 0.7998 - val_acc: 0.6794
Epoch 7/15
 - 14s - loss: 0.5595 - acc: 0.7999 - val_loss: 0.8392 - val_acc: 0.6571
Epoch 00007: early stopping
14958/14958 [==============================] - 2s 159us/step
Validation Accuracy: 65.7107%
Validation Loss: 0.8391949456015251
Test Accuracy: 0.7828840771836795
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {

Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 1.1419 - acc: 0.5378 - val_loss: 1.0683 - val_acc: 0.5121
Epoch 2/15
 - 14s - loss: 1.0272 - acc: 0.5826 - val_loss: 1.0227 - val_acc: 0.5430
Epoch 3/15
 - 15s - loss: 1.0008 - acc: 0.5961 - val_loss: 1.0336 - val_acc: 0.5599
Epoch 4/15
 - 14s - loss: 1.0008 - acc: 0.5939 - val_loss: 0.9924 - val_acc: 0.5697
Epoch 5/15
 - 13s - loss: 0.9929 - acc: 0.6003 - val_loss: 0.9996 - val_acc: 0.5921
Epoch 6/15
 - 15s - loss: 0.9526 - acc: 0.6163 - val_loss: 0.9630 - val_acc: 0.5848
Epoch 7/15
 - 15s - loss: 0.9340 - acc: 0.6316 - val_loss: 0.9505 - val_acc: 0.6002
Epoch 8/15
 - 15s - loss: 0.9231 - acc: 0.6360 - val_loss: 0.9495 - val_acc: 0.5811
Epoch 9/15
 - 14s - loss: 0.9113 - acc: 0.6412 - val_loss: 0.9012 - val_acc: 0.6102
Epoch 10/15
 - 14s - loss: 0.9015 - acc: 0.6465 - val_loss: 0.9956 - val_acc: 0.5749
Epoch 11/15
 - 14s - loss: 0.9012 - acc: 0.6460 - val_loss: 0.9874 - val_acc: 0.5579
Epoch 12/15
 - 13s - los